<a href="https://colab.research.google.com/github/ahmedhesham47/Systems-Biology/blob/main/Systems_Networks_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import csv

In [62]:
pd1_network = pd.read_csv('PD_1_network.tsv', sep='\t')
pd1_degree = pd.read_csv('PD_1_degree.tsv', sep='\t')
pd1_pathways = pd.read_csv('PD_1_RCTM.tsv', sep='\t')

pdL1_network = pd.read_csv('PD_L1_network.tsv', sep='\t')
pdL1_degree = pd.read_csv('PD_L1_degree.tsv', sep='\t')
pdL1_pathways = pd.read_csv('PD_L1_RCTM.tsv', sep='\t')

ctla4_network = pd.read_csv('CTLA_4_network.tsv', sep='\t')
ctla4_degree = pd.read_csv('CTLA_4_degree.tsv', sep='\t')
ctla4_pathways = pd.read_csv('CTLA_4_RCTM.tsv', sep='\t')

cd80_network = pd.read_csv('CD80_network.tsv', sep='\t')
cd80_degree = pd.read_csv('CD80_degree.tsv', sep='\t')
cd80_pathways = pd.read_csv('CD80_RCTM.tsv', sep='\t')

mrna = pd.read_csv('mrna_cont.tsv', sep='\t')
labels = pd.read_csv('labels_df.tsv', sep='\t')

In [33]:
pd1_network = pd1_network[['#node1', 'node2']]
pdL1_network = pdL1_network[['#node1', 'node2']]
ctla4_network = ctla4_network[['#node1', 'node2']]
cd80_network = cd80_network[['#node1', 'node2']]

pd1_degree = pd1_degree[['#node', 'node_degree']]
pdL1_degree = pdL1_degree[['#node', 'node_degree']]
ctla4_degree = ctla4_degree[['#node', 'node_degree']]
cd80_degree = cd80_degree[['#node', 'node_degree']]

In [4]:
def find_common_interactions(list_of_dataframes):
    tuple_sets = []

    for df in list_of_dataframes:
        tuples = {tuple(sorted(row)) for row in df.to_numpy()}
        tuple_sets.append(tuples)

    common_tuples = set.intersection(*tuple_sets)

    return list(common_tuples)

In [29]:
def find_common_in_col(list_of_dataframes, column_name):
    columns = []

    for df in list_of_dataframes:
        if column_name in df.columns:
            columns.append(set(df[column_name]))

    common_values = set.intersection(*columns)

    return list(common_values)

In [41]:
def find_common_hubs(list_of_dataframes):

    top_nodes_sets = []

    for df in list_of_dataframes:
        sorted_df = df.sort_values(by='node_degree', ascending=False).head(10)
        top_nodes = set(sorted_df['#node'])
        top_nodes_sets.append(top_nodes)

    common_nodes = set.intersection(*top_nodes_sets)

    return list(common_nodes)

In [85]:
def split_and_transpose(expression_df, labels_df, label_column_name):
  expression_df = expression_df.set_index('Sample Identifier')
  labels_df = labels_df.set_index('Sample Identifier')

  expression_1 = expression_df.loc[labels_df['ICB Response'] == 1.0].transpose().reset_index()
  expression_1.columns.name = None
  expression_1 = expression_1.rename(columns={'index': 'Gene'})

  expression_0 = expression_df.loc[labels_df['ICB Response'] == 0.0].transpose().reset_index()
  expression_0.columns.name = None
  expression_0 = expression_0.rename(columns={'index': 'Gene'})

  return expression_1, expression_0

In [ ]:
common_inrxns = find_common_interactions([pd1_network, pdL1_network, ctla4_network, cd80_network])

print(len(common_inrxns))
print(common_inrxns)

common_df = pd.DataFrame(common_inrxns, columns=['Node1', 'Node2'])
common_df.to_csv('common_interactions.tsv', index=False, sep='\t')

In [ ]:
common_nodes = find_common_in_col([pd1_degree, pdL1_degree, ctla4_degree, cd80_degree], '#node')

print(len(common_nodes))
print(common_nodes)

common_nodes_df = pd.DataFrame(common_nodes, columns=['Common_nodes'])
common_nodes_df.to_csv('common_nodes.tsv', index=False, sep='\t')

In [ ]:
common_pathways = find_common_in_col([pd1_pathways, pdL1_pathways, ctla4_pathways, cd80_pathways], 'term description')

print(len(common_pathways))
print(common_pathways)

common_pathways_df = pd.DataFrame(common_pathways, columns=['Common_pathways'])
common_pathways_df.to_csv('common_pathways.tsv', index=False, sep='\t')

In [ ]:
common_hubs = find_common_hubs([pd1_degree, pdL1_degree, ctla4_degree, cd80_degree])

print(len(common_hubs))
print(common_hubs)

common_hubs_df = pd.DataFrame(common_hubs, columns=['Common_hubs'])
common_pathways_df.to_csv('common_hubs.tsv', index=False, sep='\t')

In [88]:
expression_responders = split_and_transpose(mrna, labels, 'ICB Response')[0]
expression_non_responders = split_and_transpose(mrna, labels, 'ICB Response')[1]

expression_responders.to_csv('expression_1.tsv', sep='\t', index=False)
expression_non_responders.to_csv('expression_0.tsv', sep='\t', index=False)